# Modal Symbolic Learning: Day 3

## Symbolic Land Cover Classification (*unimodal data*, *multimodal learning*)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.update()
Pkg.status()

  Activating project at `~/Desktop/modal-symbolic-learning-course`
    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
  No Changes to `~/Desktop/modal-symbolic-learning-course/Project.toml`
  No Changes to `~/Desktop/modal-symbolic-learning-course/Manifest.toml`


Status `~/Desktop/modal-symbolic-learning-course/Project.toml`
  [acdeb78f] Catch22 v0.4.5
  [a93c6f00] DataFrames v1.6.1
  [864edb3b] DataStructures v0.18.15
  [7806a523] DecisionTree v0.12.4
  [7073ff75] IJulia v1.24.2
  [6a3955dd] ImageFiltering v0.7.8
  [033835bb] JLD2 v0.4.38
  [23992714] MAT v0.10.6
⌅ [add582a8] MLJ v0.19.5
  [c6f25543] MLJDecisionTreeInterface v0.4.0
  [e54bda2e] ModalDecisionTrees v0.3.6
  [91a5bcdd] Plots v1.39.0
  [7b3b3b3f] Sole v0.3.1
  [b002da8f] SoleLogics v0.6.14
  [4249d9c7] SoleModels v0.5.6
  [2913bbd2] StatsBase v0.34.2
  [9a3f8284] Random
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [2]:
# Import libraries for statistics & Machine Learning
using Random
using DataFrames
using Plots
using StatsBase

using MLJ
using Sole

In [3]:
include("land-cover.jl")
data_dir = "data/"

X_df, y = LandCoverDataset(
    "Pavia University";
    window_size          = 3,
    ninstances_per_class = 40,
    pad_window_size      = 5,
);

Load LandCoverDataset: Pavia University...
window_size         = (3, 3)
pad_window_size     = (5, 5)
ninstances_per_class  = 40
ninstances_per_class_strategy = updownsampling
flattened           = false
apply_filter        = false
seed                = 1

Image size: (610, 340, 103)
class_counts_d = [("Asphalt", 1 => 6631), ("Meadows", 2 => 18649), ("Gravel", 3 => 2099), ("Trees", 4 => 3064), ("Painted metal sheets", 5 => 1345), ("Bare Soil", 6 => 5029), ("Bitumen", 7 => 1330), ("Self-Blocking Bricks", 8 => 3682), ("Shadows", 9 => 947)]
no_class_counts = 164624
n_classes = 9
ninstances = 40 * 9 = 360
effective_class_counts_d = [("Asphalt", 1 => 40), ("Meadows", 2 => 40), ("Gravel", 3 => 40), ("Trees", 4 => 40), ("Painted metal sheets", 5 => 40), ("Bare Soil", 6 => 40), ("Bitumen", 7 => 40), ("Self-Blocking Bricks", 8 => 40), ("Shadows", 9 => 40)]
countmap(labels) = Dict(5 => 40, 4 => 40, 6 => 40, 7 => 40, 2 => 40, 9 => 40, 8 => 40, 3 => 40, 1 => 40)


In [4]:
countmap(y)

Dict{String, Int64} with 9 entries:
  "Self-Blocking Bricks" => 40
  "Bitumen"              => 40
  "Gravel"               => 40
  "Bare Soil"            => 40
  "Painted metal sheets" => 40
  "Shadows"              => 40
  "Trees"                => 40
  "Asphalt"              => 40
  "Meadows"              => 40

In [5]:
length.(X_df)

Row,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,⋯
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
2,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
3,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
4,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
5,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
6,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
7,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯
9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,⋯


In [49]:
X_df = broadcast(values->Matrix{Float64}(values), X_df)

Row,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,⋯
,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,⋯
1,[1186.0 1294.0 1291.0; 1701.0 1705.0 1373.0; 1823.0 1359.0 1616.0],[1069.0 1177.0 1369.0; 1708.0 1463.0 1041.0; 2055.0 1229.0 1440.0],[1176.0 1179.0 1265.0; 1686.0 1190.0 939.0; 1967.0 1256.0 1245.0],[1318.0 1256.0 1273.0; 1528.0 1106.0 1133.0; 1829.0 1267.0 1085.0],[1278.0 1286.0 1326.0; 1414.0 1082.0 1308.0; 1759.0 1231.0 1031.0],[1183.0 1290.0 1254.0; 1436.0 1130.0 1411.0; 1740.0 1150.0 1087.0],[1120.0 1238.0 1177.0; 1423.0 1268.0 1329.0; 1731.0 1165.0 1165.0],[1199.0 1184.0 1084.0; 1346.0 1293.0 1228.0; 1745.0 1214.0 1167.0],[1296.0 1155.0 1078.0; 1261.0 1224.0 1212.0; 1757.0 1198.0 1164.0],[1299.0 1168.0 1152.0; 1306.0 1287.0 1178.0; 1767.0 1163.0 1159.0],[1252.0 1282.0 1255.0; 1342.0 1319.0 1167.0; 1725.0 1160.0 1151.0],[1258.0 1318.0 1273.0; 1356.0 1207.0 1198.0; 1733.0 1179.0 1111.0],[1264.0 1306.0 1240.0; 1359.0 1186.0 1203.0; 1793.0 1161.0 1078.0],[1233.0 1304.0 1273.0; 1356.0 1178.0 1205.0; 1827.0 1127.0 1086.0],[1277.0 1252.0 1276.0; 1346.0 1182.0 1187.0; 1790.0 1173.0 1096.0],[1303.0 1229.0 1252.0; 1343.0 1199.0 1185.0; 1795.0 1194.0 1089.0],[1300.0 1287.0 1239.0; 1349.0 1207.0 1230.0; 1813.0 1192.0 1099.0],[1313.0 1346.0 1220.0; 1343.0 1209.0 1268.0; 1827.0 1169.0 1123.0],[1339.0 1375.0 1242.0; 1361.0 1225.0 1274.0; 1832.0 1186.0 1139.0],[1339.0 1397.0 1300.0; 1386.0 1261.0 1254.0; 1802.0 1235.0 1129.0],[1361.0 1367.0 1302.0; 1386.0 1269.0 1277.0; 1775.0 1243.0 1131.0],[1412.0 1368.0 1317.0; 1364.0 1250.0 1323.0; 1773.0 1261.0 1172.0],[1442.0 1402.0 1340.0; 1380.0 1265.0 1346.0; 1813.0 1266.0 1235.0],[1442.0 1421.0 1353.0; 1444.0 1311.0 1349.0; 1828.0 1262.0 1243.0],[1439.0 1437.0 1399.0; 1505.0 1320.0 1350.0; 1810.0 1261.0 1267.0],[1465.0 1446.0 1428.0; 1521.0 1344.0 1391.0; 1812.0 1264.0 1301.0],[1496.0 1444.0 1449.0; 1480.0 1374.0 1445.0; 1819.0 1259.0 1315.0],[1492.0 1459.0 1452.0; 1452.0 1407.0 1443.0; 1824.0 1251.0 1329.0],[1506.0 1478.0 1463.0; 1474.0 1431.0 1417.0; 1816.0 1257.0 1346.0],[1548.0 1514.0 1482.0; 1477.0 1445.0 1406.0; 1809.0 1327.0 1366.0],[1603.0 1526.0 1506.0; 1480.0 1464.0 1418.0; 1815.0 1391.0 1367.0],[1601.0 1547.0 1533.0; 1492.0 1473.0 1445.0; 1813.0 1403.0 1353.0],[1583.0 1568.0 1543.0; 1533.0 1471.0 1461.0; 1786.0 1402.0 1362.0],[1616.0 1582.0 1566.0; 1572.0 1482.0 1500.0; 1782.0 1442.0 1370.0],[1632.0 1597.0 1601.0; 1571.0 1497.0 1523.0; 1800.0 1471.0 1398.0],[1627.0 1606.0 1629.0; 1578.0 1514.0 1532.0; 1796.0 1468.0 1423.0],[1642.0 1604.0 1615.0; 1609.0 1521.0 1543.0; 1814.0 1443.0 1442.0],[1655.0 1630.0 1597.0; 1610.0 1546.0 1563.0; 1833.0 1445.0 1462.0],[1652.0 1620.0 1602.0; 1592.0 1554.0 1577.0; 1806.0 1452.0 1471.0],[1665.0 1580.0 1624.0; 1593.0 1548.0 1587.0; 1776.0 1461.0 1454.0],[1677.0 1589.0 1640.0; 1617.0 1572.0 1592.0; 1786.0 1484.0 1431.0],[1684.0 1619.0 1641.0; 1643.0 1582.0 1600.0; 1785.0 1491.0 1445.0],[1698.0 1637.0 1638.0; 1645.0 1582.0 1607.0; 1777.0 1493.0 1477.0],[1694.0 1645.0 16

In [50]:
# Let's unwind the spatial axes
X_df_static = Matrix(X_df)
cols = []
for i_var in 1:size(X_df_static, 2)
    var_unroll = cat(X_df_static[:,i_var]...; dims = 3)
    append!(cols, eachrow(reshape(var_unroll, (9, nrow(X_df)))))
end
X_df_static = DataFrame(cols, ["$n[$i][$j]" for n in names(X_df) for i in 1:3 for j in 1:3])

Row,V1[1][1],V1[1][2],V1[1][3],V1[2][1],V1[2][2],V1[2][3],V1[3][1],V1[3][2],V1[3][3],V2[1][1],V2[1][2],V2[1][3],V2[2][1],V2[2][2],V2[2][3],V2[3][1],V2[3][2],V2[3][3],V3[1][1],V3[1][2],V3[1][3],V3[2][1],V3[2][2],V3[2][3],V3[3][1],V3[3][2],V3[3][3],V4[1][1],V4[1][2],V4[1][3],V4[2][1],V4[2][2],V4[2][3],V4[3][1],V4[3][2],V4[3][3],V5[1][1],V5[1][2],V5[1][3],V5[2][1],V5[2][2],V5[2][3],V5[3][1],V5[3][2],V5[3][3],V6[1][1],V6[1][2],V6[1][3],V6[2][1],V6[2][2],V6[2][3],V6[3][1],V6[3][2],V6[3][3],V7[1][1],V7[1][2],V7[1][3],V7[2][1],V7[2][2],V7[2][3],V7[3][1],V7[3][2],V7[3][3],V8[1][1],V8[1][2],V8[1][3],V8[2][1],V8[2][2],V8[2][3],V8[3][1],V8[3][2],V8[3][3],V9[1][1],V9[1][2],V9[1][3],V9[2][1],V9[2][2],V9[2][3],V9[3][1],V9[3][2],V9[3][3],V10[1][1],V10[1][2],V10[1][3],V10[2][1],V10[2][2],V10[2][3],V10[3][1],V10[3][2],V10[3][3],V11[1][1],V11[1][2],V11[1][3],V11[2][1],V11[2][2],V11[2][3],V11[3][1],V11[3][2],V11[3][3],V12[1][1],⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1186.0,1701.0,1823.0,1294.0,1705.0,1359.0,1291.0,1373.0,1616.0,1069.0,1708.0,2055.0,1177.0,1463.0,1229.0,1369.0,1041.0,1440.0,1176.0,1686.0,1967.0,1179.0,1190.0,1256.0,1265.0,939.0,1245.0,1318.0,1528.0,1829.0,1256.0,1106.0,1267.0,1273.0,1133.0,1085.0,1278.0,1414.0,1759.0,1286.0,1082.0,1231.0,1326.0,1308.0,1031.0,1183.0,1436.0,1740.0,1290.0,1130.0,1150.0,1254.0,1411.0,1087.0,1120.0,1423.0,1731.0,1238.0,1268.0,1165.0,1177.0,1329.0,1165.0,1199.0,1346.0,1745.0,1184.0,1293.0,1214.0,1084.0,1228.0,1167.0,1296.0,1261.0,1757.0,1155.0,1224.0,1198.0,1078.0,1212.0,1164.0,1299.0,1306.0,1767.0,1168.0,1287.0,1163.0,1152.0,1178.0,1159.0,1252.0,1342.0,1725.0,1282.0,1319.0,1160.0,1255.0,1167.0,1151.0,1258.0,⋯
2,1153.0,784.0,1173.0,1201.0,1618.0,1071.0,1563.0,1508.0,1434.0,1024.0,1024.0,1110.0,1258.0,1548.0,1152.0,1480.0,1368.0,1193.0,1077.0,1147.0,1143.0,1224.0,1320.0,1276.0,1397.0,1266.0,1164.0,1130.0,1165.0,1233.0,1124.0,1220.0,1326.0,1341.0,1282.0,1345.0,1245.0,1122.0,1329.0,1214.0,1311.0,1316.0,1330.0,1406.0,1519.0,1325.0,1163.0,1368.0,1287.0,1357.0,1309.0,1352.0,1463.0,1630.0,1388.0,1184.0,1346.0,1351.0,1391.0,1323.0,1362.0,1468.0,1645.0,1342.0,1146.0,1268.0,1401.0,1392.0,1324.0,1351.0,1367.0,1563.0,1303.0,1195.0,1296.0,1457.0,1446.0,1383.0,1300.0,1371.0,1477.0,1330.0,1260.0,1379.0,1497.0,1456.0,1477.0,1291.0,1460.0,1466.0,1421.0,1252.0,1443.0,1442.0,1394.0,1541.0,1359.0,1501.0,1466.0,1436.0,⋯
3,880.0,672.0,540.0,804.0,865.0,757.0,727.0,1067.0,803.0,726.0,498.0,676.0,486.0,629.0,886.0,771.0,957.0,817.0,561.0,401.0,791.0,509.0,548.0,784.0,648.0,808.0,733.0,579.0,536.0,769.0,535.0,768.0,773.0,759.0,714.0,669.0,623.0,608.0,722.0,651.0,809.0,758.0,684.0,736.0,729.0,690.0,570.0,740.0,661.0,701.0,654.0,551.0,763.0,694.0,688.0,615.0,765.0,627.0,613.0,602.0,530.0,713.0,722.0,644.0,658.0,729.0,670.0,663.0,705.0,552.0,684.0,700.0,669.0,675.0,655.0,688.0,724.0,802.0,622.0,623.0,706.0,728.0,668.0,647.0,694.0,702.0,824.0,671.0,552.0,695.0,753.0,649.0,684.0,663.0,732.0,759.0,756.0,629.0,658.0,744.0,⋯
4,1454.0,512.0,1430.0,1082.0,985.0,1400.0,1291.0,936.0,1098.0,1178.0,741.0,1124.0,876.0,1014.0,1259.0,1229.0,1064.0,1282.0,952.0,1074.0,1149.0,881.0,901.0,1249.0,1201.0,1014.0,1286.0,1071.0,1251.0,1238.0,1034.0,1039.0,1215.0,1198.0,996.0,1107.0,1162.0,1243.0,12

In [51]:
using SoleData

X_multimodal = MultiModalDataset([X_df, X_df_static])

● MultiModalDataset{DataFrame}
   └─ dimensionalities: (2, 0)
- Modality 1 / 2
   └─ dimensionality: 2
360×103 SubDataFrame
 Row │ V1                                 V2                                 V ⋯
     │ Array…                             Array…                             A ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [1186.0 1294.0 1291.0; 1701.0 17…  [1069.0 1177.0 1369.0; 1708.0 14…  [ ⋯
   2 │ [1153.0 1201.0 1563.0; 784.0 161…  [1024.0 1258.0 1480.0; 1024.0 15…  [
   3 │ [880.0 804.0 727.0; 672.0 865.0 …  [726.0 486.0 771.0; 498.0 629.0 …  [
   4 │ [1454.0 1082.0 1291.0; 512.0 985…  [1178.0 876.0 1229.0; 741.0 1014…  [
   5 │ [1603.0 1091.0 594.0; 1003.0 126…  [1602.0 1092.0 689.0; 804.0 1240…  [ ⋯
   6 │ [966.0 632.0 605.0; 482.0 669.0 …  [1171.0 630.0 451.0; 446.0 507.0…  [
   7 │ [1219.0 1459.0 1247.0; 1377.0 48…  [1377.0 1259.0 1255.0; 1298.0 88…  [
   8 │ [510.0 686.0 863.0; 637.0 748.0 …  [365.0 726.0 761.0; 810.0 639.0 … 

In [52]:
using ModalDecisionTrees

model = ModalDecisionTree(; relations = :RCC8)

ModalDecisionTree(
  max_depth = nothing, 
  min_samples_leaf = 4, 
  min_purity_increase = 0.002, 
  max_purity_at_leaf = Inf, 
  max_modal_depth = nothing, 
  relations = :RCC8, 
  features = nothing, 
  conditions = nothing, 
  featvaltype = Float64, 
  initconditions = nothing, 
  downsize = ModalDecisionTrees.MLJInterface.var"#downsize#43"(), 
  print_progress = false, 
  rng = Random._GLOBAL_RNG(), 
  display_depth = nothing, 
  min_samples_split = nothing, 
  n_subfeatures = identity, 
  post_prune = false, 
  merge_purity_threshold = nothing, 
  feature_importance = :split)

In [28]:
# Train in cross-validation!
e = @time evaluate!(machine(model, X_multimodal, y);
    resampling=StratifiedCV(rng = Random.Xoshiro(1), shuffle=true, nfolds = 10),
    measures=[accuracy],
    verbosity=0,
    check_measure=false
)

[ Info: Precomputing logiset...
┌ Warning: ScalarOneStepMemoset: Found globalrel in relations in a single-world case.
└ @ SoleModels ~/.julia/packages/SoleModels/xvfwj/src/logisets/scalar/onestep-memoset.jl:304


694.598215 seconds (7.75 G allocations: 729.113 GiB, 11.43% gc time, 12.83% compilation time: <1% of which was recompilation)


PerformanceEvaluation object with these fields:
  model, measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows, resampling, repeats
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.775       │ 0.0537  │ [0.778, 0.833, 0.611, 0. ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [29]:
# Test accuracies per fold
e.per_fold

1-element Vector{Vector{Float64}}:
 [0.7777777777777778, 0.8333333333333334, 0.6111111111111112, 0.9166666666666666, 0.7777777777777778, 0.7777777777777778, 0.6944444444444444, 0.75, 0.7777777777777778, 0.8333333333333334]

In [30]:
trees = map((((train_idxs, test_idxs), rep),)->begin
    predictions, tree_test = rep.sprinkle(slicedataset(X_multimodal, test_idxs), y[test_idxs]; simplify = true)
    tree_test
end, zip(e.train_test_rows, e.report_per_fold))

10-element Vector{DecisionTree{String, MultiFormula{Atom{ScalarCondition{Int64, SoleModels.UnivariateValue, ScalarMetaCondition{SoleModels.UnivariateValue, typeof(>=)}}}}, SoleModels.ConstantModel{String}}}:
 ▣ {2}(V923 ≥ 1714)
├✔ {2}(V52 ≥ 1122)
│├✔ {2}(V248 ≥ 1882)
││├✔ Painted metal sheets
││└✘ {1}(⟨G⟩(min[V47] ≥ 2103))
││ ├✔ Gravel
││ └✘ {2}(V178 ≥ 1476)
││  ├✔ Self-Blocking Bricks
││  └✘ {1}(⟨G⟩(min[V2] < 1021))
││   ├✔ Self-Blocking Bricks
││   └✘ Gravel
│└✘ {1}(⟨G⟩(min[V68] < 563))
│ ├✔ Trees
│ └✘ {1}(⟨G⟩(min[V29] ≥ 1347))
│  ├✔ Bare Soil
│  └✘ {2}(V662 ≥ 1915)
│   ├✔ Meadows
│   └✘ {2}(V63 ≥ 579)
│    ├✔ Meadows
│    └✘ Bare Soil
└✘ {1}(⟨G⟩(min[V6] < 567))
 ├✔ Shadows
 └✘ {2}(V593 ≥ 1343)
  ├✔ {2}(V917 ≥ 1490)
  │├✔ Gravel
  │└✘ {1}(⟨G⟩(min[V3] < 754))
  │ ├✔ Asphalt
  │ └✘ Bitumen
  └✘ Asphalt

 ▣ {2}(V923 ≥ 1714)
├✔ {2}(V44 ≥ 1109)
│├✔ {1}(⟨G⟩(min[V38] ≥ 2424))
││├✔ Painted metal sheets
││└✘ {1}(⟨G⟩(min[V67] ≥ 2152))
││ ├✔ Gravel
││ └✘ {2}(V743 ≥ 1919)
││  ├✔ Self-Blocking Br

In [31]:
rules = vcat(listrules.(trees)...)

134-element Vector{Rule{String, A, SoleModels.ConstantModel{String}} where A<:Formula}:
 ▣ {2}(V248 ≥ 1882)  🠮  Painted metal sheets

 ▣ {1}(⟨G⟩(min[V47] ≥ 2103)) ∧ {2}(V52 ≥ 1122) ∧ (V248 < 1882)  🠮  Gravel

 ▣ {1}([G](min[V47] < 2103)) ∧ {2}(V178 ≥ 1476) ∧ (V248 < 1882)  🠮  Self-Blocking Bricks

 ▣ {1}⟨G⟩(min[V2] < 1021) ∧ [G](min[V47] < 2103) ∧ {2}(V52 ≥ 1122) ∧ (V248 < 1882) ∧ (V178 < 1476)  🠮  Self-Blocking Bricks

 ▣ {1}[G](min[V47] < 2103) ∧ [G](min[V2] ≥ 1021) ∧ {2}(V52 ≥ 1122) ∧ (V248 < 1882) ∧ (V178 < 1476)  🠮  Gravel

 ▣ {1}(⟨G⟩(min[V68] < 563)) ∧ {2}(V923 ≥ 1714) ∧ (V52 < 1122)  🠮  Trees

 ▣ {1}⟨G⟩(min[V29] ≥ 1347) ∧ [G](min[V68] ≥ 563) ∧ {2}(V923 ≥ 1714) ∧ (V52 < 1122)  🠮  Bare Soil

 ▣ {1}[G](min[V68] ≥ 563) ∧ [G](min[V29] < 1347) ∧ {2}(V662 ≥ 1915) ∧ (V52 < 1122)  🠮  Meadows

 ▣ {1}[G](min[V68] ≥ 563) ∧ [G](min[V29] < 1347) ∧ {2}(V63 ≥ 579) ∧ (V52 < 1122) ∧ (V662 < 1915)  🠮  Meadows

 ▣ {1}[G](min[V68] ≥ 563) ∧ [G](min[V29] < 1347) ∧ {2}(V923 ≥ 1714) ∧ (V52 < 1122) ∧ (V6

In [32]:
# Every symbolic model (including rules) can have has additional information attached
println(rules[1])

ruleinfo = SoleModels.info(rules[1])
println(keys(ruleinfo))

▣ {2}(V248 ≥ 1882)  🠮  Painted metal sheets

(:supporting_labels, :supporting_predictions, :shortform)


In [33]:
ruleinfo[:supporting_predictions] |> length

4

In [34]:
sort(readmetrics.(rules), by=x->x[:coverage], rev = true)

134-element Vector{NamedTuple{(:ninstances, :confidence, :coverage), Tuple{Int64, Float64, Float64}}}:
 (ninstances = 7, confidence = 0.57, coverage = 0.19)
 (ninstances = 6, confidence = 0.67, coverage = 0.17)
 (ninstances = 6, confidence = 0.67, coverage = 0.17)
 (ninstances = 6, confidence = 0.67, coverage = 0.17)
 (ninstances = 6, confidence = 0.67, coverage = 0.17)
 (ninstances = 5, confidence = 0.4, coverage = 0.14)
 (ninstances = 5, confidence = 0.4, coverage = 0.14)
 (ninstances = 5, confidence = 0.8, coverage = 0.14)
 (ninstances = 5, confidence = 0.6, coverage = 0.14)
 (ninstances = 5, confidence = 0.8, coverage = 0.14)
 (ninstances = 5, confidence = 0.8, coverage = 0.14)
 (ninstances = 5, confidence = 0.6, coverage = 0.14)
 (ninstances = 5, confidence = 0.6, coverage = 0.14)
 ⋮
 (ninstances = 1, confidence = 0.0, coverage = 0.03)
 (ninstances = 1, confidence = 0.0, coverage = 0.03)
 (ninstances = 1, confidence = 1.0, coverage = 0.03)
 (ninstances = 1, confidence = 0.0, cover

In [53]:
# goodrules = filter(r->readmetrics(r)[:ninstances] > 1, rules)
goodrules = sort(rules, by=r->readmetrics(r)[:coverage], rev = true)
printmodel.(goodrules; show_metrics = true, threshold_digits = 4);

▣ {2}(V52 ≥ 1114.0) ∧ (V248 < 1882.0)  🠮  Self-Blocking Bricks : (ninstances = 7, confidence = 0.57, coverage = 0.19)
▣ {1}⟨G⟩(min[V44] < 543.0) ∧ [G]((min[V44] < 543.0) → [N̅T̅P̅P̅](min[V28] < 620.0)) ∧ {2}(V923 ≥ 1714.0) ∧ (V44 < 1109.0)  🠮  Trees : (ninstances = 6, confidence = 0.67, coverage = 0.17)
▣ {2}(V540 ≥ 428.0) ∧ (V52 < 1114.0)  🠮  Meadows : (ninstances = 6, confidence = 0.67, coverage = 0.17)
▣ {2}(V52 ≥ 1114.0) ∧ (V248 < 1882.0)  🠮  Self-Blocking Bricks : (ninstances = 6, confidence = 0.67, coverage = 0.17)
▣ {1}([G](min[V6] ≥ 567.0)) ∧ {2}(V593 ≥ 1343.0) ∧ (V923 < 1714.0)  🠮  Bitumen : (ninstances = 6, confidence = 0.67, coverage = 0.17)
▣ {1}[G](min[V68] ≥ 563.0) ∧ [G](min[V29] < 1347.0) ∧ {2}(V923 ≥ 1714.0) ∧ (V52 < 1114.0) ∧ (V877 < 2483.0)  🠮  Bare Soil : (ninstances = 5, confidence = 0.4, coverage = 0.14)
▣ {1}([G](min[V48] < 2114.0)) ∧ {2}(V197 ≥ 1459.0) ∧ (V248 < 1882.0)  🠮  Self-Blocking Bricks : (ninstances = 5, confidence = 0.4, coverage = 0.14)
▣ {1}(⟨G⟩(min[V

▣ {1}(⟨G⟩((min[V44] < 543.0) ∧ ⟨N̅T̅P̅P̅⟩(min[V28] ≥ 620.0))) ∧ {2}(V923 ≥ 1714.0) ∧ (V44 < 1109.0)  🠮  Meadows : (ninstances = 2, confidence = 1.0, coverage = 0.06)
▣ {1}⟨G⟩(min[V4] < 1185.0) ∧ [G](min[V38] < 2267.0) ∧ {2}(V167 ≥ 1475.0)  🠮  Self-Blocking Bricks : (ninstances = 2, confidence = 0.5, coverage = 0.06)
▣ {1}([G](min[V38] < 2267.0)) ∧ {2}(V52 ≥ 1114.0) ∧ (V167 < 1475.0)  🠮  Gravel : (ninstances = 2, confidence = 0.0, coverage = 0.06)
▣ {1}(⟨G⟩(min[V68] < 563.0)) ∧ {2}(V923 ≥ 1714.0) ∧ (V52 < 1114.0)  🠮  Trees : (ninstances = 2, confidence = 1.0, coverage = 0.06)
▣ {1}⟨G⟩(min[V29] ≥ 1347.0) ∧ [G](min[V68] ≥ 563.0) ∧ {2}(V923 ≥ 1714.0) ∧ (V52 < 1114.0)  🠮  Bare Soil : (ninstances = 2, confidence = 0.5, coverage = 0.06)
▣ {1}([G](min[V5] ≥ 516.0)) ∧ {2}(V917 ≥ 1490.0) ∧ (V923 < 1714.0) ∧ (V106 < 1346.0)  🠮  Gravel : (ninstances = 2, confidence = 1.0, coverage = 0.06)
▣ {1}([G](min[V1] < 2642.0)) ∧ {2}(V248 ≥ 1882.0)  🠮  Bare Soil : (ninstances = 2, confidence = 1.0, coverage 